# Group cross sections from OpenMC

- Date: 21 march 2023
- Based on: Zsolts Serpent groupxs files
- Changelog:
    - 21 march 2023: Initial version

### Questions
1. ...

In [11]:
# Fuel properties
FUEL_TEMP = 900 # K
# FUEL_ENRICHMENT = 4.0 # wo% # NOT NEEDED IN THIS MODEL
# FUEL_R = 5 # cm # NOT NEEDED IN THIS MODEL

# Moderator properties
MODERATOR_TEMP = 600 # K
MODERATOR_R = 1.15 # cm

# Cladding properties
CLADDING_TEMP = 900 # K
CLADDING_R = 1.25 # cm

# Lattice properties
LATTICE_ELEMENTS_INNER = 7
LATTICE_PITCH_INNER = 3 # cm

# Plenum properties
PLENUM_HEIGHT = 25 # cm

# Core properties
CORE_HEIGHT = 350 # cm

# Run properties
INACTIVE_BATCHES = 50
PARTICLE_COUNT = 30000
ACTIVE_BATCH_COUNT = 300
FAST_REACTOR = True # True if epithermal, False if thermal

DO_PLOT = True

In [13]:
import openmc
import numpy as np
import matplotlib.pyplot as plt
import os
from tools import run_tools, nuclear_data_tools
import time
import random
import openmc.mgxs as mgxs

FUEL_ONLY = False

for FAST_REACTOR in [True, False]:
    # FAST_REACTOR = False # True if epithermal, False if thermal
    NEUTRON_TEMP = "EPITHERMAL" if FAST_REACTOR else "THERMAL"
    print(NEUTRON_TEMP)

    # @@@@@@@@@@@@@@@@@@@@@@ Materials @@@@@@@@@@@@@@@@@@@@@@
    # Fuel
    material_fuel = openmc.Material(1, "ULiF", temperature=FUEL_TEMP)
    material_fuel.add_nuclide('U235', 3.11e-2, 'wo')
    material_fuel.add_nuclide('U238', 5.909e-1, 'wo')
    material_fuel.add_nuclide('Li7', 4.8358e-2, 'wo')
    material_fuel.add_nuclide('Li6', 2.418e-6, 'wo')
    material_fuel.add_nuclide('F19', 3.2964e-1, 'wo')
    material_fuel.set_density('g/cm3', 5.01)

    # Cladding
    material_cladding = openmc.Material(3, "CSi", temperature=CLADDING_TEMP)
    material_cladding.add_nuclide('C0', 2.9936e-1, 'wo')
    material_cladding.add_nuclide('Si28', 6.4365e-1, 'wo')
    material_cladding.add_nuclide('Si29', 3.3866e-2, 'wo')
    material_cladding.add_nuclide('Si30', 2.3120e-2, 'wo')
    material_cladding.set_density('g/cm3', 3.21)

    # Moderator
    material_moderator = openmc.Material(2, "ZrH", temperature=MODERATOR_TEMP)
    material_moderator.add_nuclide('Zr90', 4.9793e-1, 'wo')
    material_moderator.add_nuclide('Zr91', 1.0980e-1, 'wo')
    material_moderator.add_nuclide('Zr92', 1.6967e-1, 'wo')
    material_moderator.add_nuclide('Zr94', 1.7569e-1, 'wo')
    material_moderator.add_nuclide('Zr96', 2.8908e-2, 'wo')
    material_moderator.add_nuclide('H1', 1.8007e-2, 'wo')
    material_moderator.add_nuclide('H2', 4.1389e-6, 'wo')
    material_moderator.set_density('g/cm3', 5.66)

    material_moderator.add_s_alpha_beta('c_H_in_ZrH') # DID NOT EXIST AT OR NEAR 900 K

    # Create Materials list and export to XML
    mats = openmc.Materials([material_fuel, material_cladding, material_moderator])
    mats.export_to_xml()
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



    # @@@@@@@@@@@@@@@@@@@@@@ Geometry @@@@@@@@@@@@@@@@@@@@@@
    # Planes
    plane_bottom = openmc.ZPlane(z0=-CORE_HEIGHT/2, boundary_type='vacuum')
    plane_top = openmc.ZPlane(z0=CORE_HEIGHT/2, boundary_type='vacuum')

    # Cylinders
    r_moderator = openmc.ZCylinder(r=MODERATOR_R)
    r_cladding = openmc.ZCylinder(r=CLADDING_R)

    # Regions
    region_height_bound = +plane_bottom & -plane_top
    region_moderator = -r_moderator & region_height_bound
    region_cladding = +r_moderator & -r_cladding & region_height_bound
    region_fuel = +r_cladding & region_height_bound

    # # Cells
    cell_cladding = openmc.Cell(name="cladding", fill=material_cladding, region=region_cladding)
    cell_moderator = openmc.Cell(name="moderator", fill=material_moderator, region=region_moderator)
    cell_fuel = openmc.Cell(name="fuel", fill=material_fuel, region=region_fuel)

    # # Universe
    universe_pin = openmc.Universe(cells=(cell_cladding, cell_moderator, cell_fuel))

    # # Outer universe
    cell_all_fuel = openmc.Cell(name="outer", fill=material_fuel)
    universe_outer = openmc.Universe(cells=(cell_all_fuel,))
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



    # @@@@@@@@@@@@@@@@@@@@@@ Lattice @@@@@@@@@@@@@@@@@@@@@@
    # Inner lattice
    innerLatticeDistanceToCenter = LATTICE_PITCH_INNER * LATTICE_ELEMENTS_INNER / 2
    lattice_inner = openmc.RectLattice()
    lattice_inner.lower_left = (-innerLatticeDistanceToCenter, -innerLatticeDistanceToCenter) # TODO: Is this needed?
    # lattice_inner.lower_left = (0, 0)
    lattice_inner.pitch = (LATTICE_PITCH_INNER, LATTICE_PITCH_INNER)

    if not FAST_REACTOR: # Thermal
        inner_lattice_string =   """F F F F F F F
                                    F M M M M M F
                                    F M M M M M F
                                    F M M M M M F
                                    F M M M M M F
                                    F M M M M M F
                                    F F F F F F F"""
    else: # Epithermal
        inner_lattice_string =   """F F F F F F F
                                    F F F F F F F
                                    F F M F M F F
                                    F F F F F F F
                                    F F M F M F F
                                    F F F F F F F
                                    F F F F F F F"""
        
    if FUEL_ONLY:
        inner_lattice_string =   """F F F F F F F
                                    F F F F F F F
                                    F F F F F F F
                                    F F F F F F F
                                    F F F F F F F
                                    F F F F F F F
                                    F F F F F F F"""
        
    lattice_inner.universes = [[universe_outer if cell == 'F' else universe_pin for cell in row.split()]
                                for row in inner_lattice_string.splitlines()]

    # Make a universe out of the inner lattice
    cell_lattice_inner = openmc.Cell(name="lattice_inner", fill=lattice_inner)
    universe_lattice_inner = openmc.Universe(cells=(cell_lattice_inner,))

    plane_plenum_bottom = openmc.ZPlane(z0=-CORE_HEIGHT/2+PLENUM_HEIGHT)
    plane_plenum_top = openmc.ZPlane(z0=CORE_HEIGHT/2-PLENUM_HEIGHT)

    # Make planes on the west, east, north, and south sides of the lattice with reflective boundary conditions
    plane_west = openmc.XPlane(x0=-innerLatticeDistanceToCenter, boundary_type='reflective')
    plane_east = openmc.XPlane(x0=innerLatticeDistanceToCenter, boundary_type='reflective')
    plane_south = openmc.YPlane(y0=-innerLatticeDistanceToCenter, boundary_type='reflective')
    plane_north = openmc.YPlane(y0=innerLatticeDistanceToCenter, boundary_type='reflective')

    region_within_lattice = +plane_west & -plane_east & +plane_south & -plane_north

    bottom_slab = +plane_bottom & -plane_plenum_bottom
    top_slab = -plane_top & +plane_plenum_top
    region_height_bound_inner = +plane_plenum_bottom & -plane_plenum_top
    # region_containment = (+r_containment_inner & -r_containment_outer & region_height_bound)
    region_plenum = region_within_lattice & (top_slab | bottom_slab)
    region_main = region_within_lattice & region_height_bound_inner

    # cell_containment = openmc.Cell(name="containment", fill=material_cladding, region=region_containment)
    cell_main = openmc.Cell(fill=lattice_inner, region=region_main)
    cell_plenum = openmc.Cell(name="plenum", fill=material_fuel, region=region_plenum)
    geometry = openmc.Geometry([cell_main, cell_plenum])
    geometry.export_to_xml()
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



    # @@@@@@@@@@@@@@@@@@@@@@ Plots @@@@@@@@@@@@@@@@@@@@@@
    if DO_PLOT:
        PLOT_COLOR_BY = 'material'
        PLOT_PIXELS = (1000, 1000)
        PLOT_FUEL_COLOR = 'yellowgreen'
        PLOT_CLADDING_COLOR = 'lightslategray'
        PLOT_MODERATOR_COLOR = 'midnightblue'

        colors = {
            material_fuel: PLOT_FUEL_COLOR,
            material_cladding: PLOT_CLADDING_COLOR,
            material_moderator: PLOT_MODERATOR_COLOR
        }

        slize_coord_XY = 0

        plotXY = openmc.Plot.from_geometry(geometry, basis='xy', slice_coord=slize_coord_XY)
        plotXY.color_by = PLOT_COLOR_BY
        plotXY.filename = f'plotXY-{NEUTRON_TEMP}'
        if FUEL_ONLY:
            plotXY.filename = f'plotXY-FUEL_ONLY'
        plotXY.pixels = PLOT_PIXELS
        plotXY.colors = colors
        plotXY.to_ipython_image()

        # Add slice_coord to actually see the moderator rods in the plot
        plotXZ = openmc.Plot.from_geometry(geometry, basis='xz', slice_coord=4)
        plotXZ.color_by = PLOT_COLOR_BY
        plotXZ.filename = f'plotXZ-{NEUTRON_TEMP}'
        if FUEL_ONLY:
            plotXZ.filename = f'plotXZ-FUEL_ONLY'
        plotXZ.pixels = PLOT_PIXELS
        plotXZ.colors = colors
        plotXZ.to_ipython_image()
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


    USE_SAMPLED_DATA = False
    # @@@@@@@@@@@@@@@@@@@@@@ Perturbations @@@@@@@@@@@@@@@@@@@@@@
    nuclear_data_tools.restore_F19_h5()
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


    # @@@@@@@@@@@@@@@@@@@@@@ Settings @@@@@@@@@@@@@@@@@@@@@@
    point = openmc.stats.Point((0, 0, 0))
    src = openmc.Source(space=point)

    # # Shannon Entropy
    # regular_mesh = openmc.RegularMesh()
    # regular_mesh.lower_left, regular_mesh.upper_right = geometry.bounding_box
    # regular_mesh.dimension = [20, 20, 20]

    settings = openmc.Settings()
    settings.seed = random.randint(1, 1e10)
    settings.source = src
    # settings.entropy_mesh = regular_mesh
    settings.batches = ACTIVE_BATCH_COUNT + INACTIVE_BATCHES
    settings.inactive = INACTIVE_BATCHES
    settings.particles = PARTICLE_COUNT
    settings.export_to_xml()
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



    # @@@@@@@@@@@@@@@@@@@@@@ Run OpenMC @@@@@@@@@@@@@@@@@@@@@@
    os.system('rm s*h5') # Remove old files
    t_start = time.time()
    openmc.run()
    t_end = time.time()
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



    # @@@@@@@@@@@@@@@@@@@@@@ Save data @@@@@@@@@@@@@@@@@@@@@@
    run_settings = {
        "fuel_only": FUEL_ONLY,
        "fast_reactor": FAST_REACTOR,
        "neutron_temp": NEUTRON_TEMP,
        "use_sampled_data": USE_SAMPLED_DATA,
        "particle_count": PARTICLE_COUNT,
        "active_batch_count": ACTIVE_BATCH_COUNT,
        "inactive_batch_count": INACTIVE_BATCHES,
        "t_start": round(t_start, 3),
        "t_end": round(t_end, 3),
        "t_elapsed": round(t_end - t_start, 3),
        "seed": settings.seed
    }
    run_tools.dump_run_settings_json(run_settings)
    run_tools.save_run_data()
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

EPITHERMAL


/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)


THERMAL


/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
